# Sequence Extraction

From `annotations.db` we extract exon information
- acceptor position
- donor position
- chromosome, transcript associated

From the `fasta` file we obtain sequences by chromosome

In [2]:
import os
import pandas as pd

from collections import (
    defaultdict,
    namedtuple
)

from src import SequencerSpliceJunction

In [3]:
path_db = "data/annotation-004.db"
path_fa = "data/Homo_sapiens-005.fa"

In [ ]:
train_chromosome = ['2','4','6','8','10','11','12','13','14','15','16','17','18','19','20','21','22','X','Y']
test_chromosome = ['1','3','5','7','9']
chromosomes = train_chromosome + test_chromosome
for chromosome in chromosomes: 
    print(chromosome)
    # initialize sequencer and load seq data
    ds = SequencerSpliceJunction(
        path_annotations = path_db, 
        path_fasta = path_fa, 
        chromosomes = chromosomes,
        len_samples = 60,
        k  = 10, # samples per exon (negative) / splice-junction (positive)
    )

    # Save exoninfo for the chromosome
    if not os.path.exists("data/exoninfo_chr_{}.csv".format(chromosome)):
        df_exon = pd.DataFrame(ds.exon_info).to_csv("data/exoninfo_chr_{}.csv".format(chromosome))
    
    if not os.path.exists("data/positive_samples_chr_{}.csv".format(chromosome)):
        ds.generate_positive_samples_fast(chromosome=chromosome,)
        
    if not os.path.exists("data/negative_samples_chr_{}.csv".format(chromosome)):
        ds.generate_negative_samples(chromosome=chromosome, allow_empty=True, save=True)


2
loading annotations
get genes id


  0%|          | 2/60675 [00:00<1:08:32, 14.75it/s]

get exon info


100%|██████████| 60675/60675 [03:14<00:00, 312.55it/s] 
0it [00:00, ?it/s]

collecting sequence by chromosome


...

  0%|          | 2/60675 [00:00<1:01:45, 16.38it/s]

get exon info


100%|██████████| 60675/60675 [03:05<00:00, 326.96it/s] 
0it [00:00, ?it/s]

collecting sequence by chromosome


16it [01:30,  6.49s/it]

___

In [ ]:
# train_chromosome = ['2','4','6','8','10','11','12','13','14','15','16','17','18','19','20','21','22','X','Y']
# test_chromosome = ['1','3','5','7','9']
# chromosomes = train_chromosome + test_chromosome
chromosomes = ["X"]
chromosomes.sort(reverse=True)

ds = SequencerSpliceJunction(
    path_annotations = path_db, 
    path_fasta = path_fa, 
    chromosomes = chromosomes,
    len_samples = 60,
    k  = 10, # samples per exon (negative) / splice-junction (positive)
)

## Exon Info: for selected chromosomes

In [ ]:
df_exon = pd.DataFrame(ds.exon_info)
df_exon.head()

In [ ]:
df_exon.shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df_exon["transcript_id"].value_counts().plot.hist(bins=50, title="Histogram: transcripts per number of exons")
plt.show()

In [ ]:
df_exon.head()

## Length of exons
- Most of the exons (75%) has length lower than 200. 
- Negative samples depends on the length of the exons. 
- Positive samples depends on the length of two exons. 

In [ ]:
df_exon["len_exon"] = df_exon.apply(lambda r: r["donor_idx"]-r["acceptor_idx"], axis=1)

In [ ]:
df_exon["len_exon"].describe()

In [ ]:
df_exon.len_exon[df_exon.len_exon<1200].plot.hist(bins=100, title="Lenght of exons")

In [ ]:
df_exon.head()

## Save exoninfo

In [ ]:
df_exon.to_csv("data/exoninfo.csv")

___ 

# Sampling of sequences
For each transcript in `.exon_info`:
1. If it contains 2 or more exons, go to step 2.
2. For each exon $e$ in the transcript sample negative cases (I discarded the transcripts with one exon)
3. For each consecutive pair of exons $e_i, e_{i+1}$ sample positive cases

### Positive Samples

In [ ]:
for chromosome in chromosomes: 
    print(chromosome)
    ds.generate_positive_samples_fast(chromosome=chromosome,)

In [ ]:
len(ds.pos_samples)

In [ ]:
# for chromosome in chromosomes: 
#     print(chromosome)
#     ds.generate_positive_samples(chromosome=chromosome, save=True)

In [ ]:
len(ds.pos_samples), ds.counter_pos

### take a look to positive samples

In [ ]:
sample = ds.pos_samples[1]
sample

In [ ]:
list(filter(lambda info: info.id_exoninfo in [sample.id_exoninfo_acceptor, sample.id_exoninfo_donor] , ds.exon_info))

### Negative Samples

In [ ]:
for chromosome in chromosomes:
    print(chromosome)
    ds.generate_negative_samples(chromosome=chromosome, allow_empty=True, save=False)

In [ ]:
len(ds.neg_samples)

### take a look to negative samples

In [ ]:
sample = ds.neg_samples[20]
sample

In [ ]:
list(filter(lambda info: info.id_exoninfo == sample.id_exoninfo , ds.exon_info))